In [ ]:
import os
%pwd

In [ ]:
os.chdir('../')
%pwd

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    training_org_img_folder: Path
    training_segment_img_folder: Path
    testing_org_img_folder: Path
    testing_segment_img_folder: Path
    number_of_classes: int

In [4]:


from src.imageSegmentation.constants import *
from src.imageSegmentation.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        evaluation_config = ModelEvaluationConfig(
            model_path = Path(config.model_path),
            training_org_img_folder= Path(config.training_org_img_folder),
            training_segment_img_folder = Path(config.training_segment_img_folder),
            testing_org_img_folder = Path(config.testing_org_img_folder),
            testing_segment_img_folder = Path(config.testing_segment_img_folder),
            number_of_classes = self.params.CLASSES
        )
        return evaluation_config

In [5]:


from src.imageSegmentation.constants import *
from src.imageSegmentation.utils.common import read_yaml, create_directories, save_json
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import cv2
import numpy as np
from tensorflow import keras
import segmentation_models as sm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img, img_to_array
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, BatchNormalization, UpSampling2D, Concatenate, concatenate, Conv2DTranspose
from keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical


def bce_jaccard_loss(y_train, y_valid, smooth=1e-7):
    y_train_flat = K.flatten(y_train)
    y_valid_flat = K.flatten(y_valid)

    bce_loss = K.binary_crossentropy(y_train_flat, y_valid_flat)

    intersection = K.sum(y_train_flat * y_valid_flat)
    union = K.sum(y_train_flat) + K.sum(y_valid_flat) - intersection
    jaccard_loss = 1.0 - (intersection + smooth) / (union + smooth)
    total_loss = bce_loss + jaccard_loss
    return total_loss

def iou_score(y_train, y_valid, smooth=1e-7):

    y_train_flat = K.flatten(y_train)
    y_valid_flat = K.flatten(y_valid)

    intersection = K.sum(y_train_flat * y_valid_flat)
    union = K.sum(y_train_flat) + K.sum(y_valid_flat) - intersection

    iou = (intersection + smooth) / (union + smooth)
    return iou



class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def convert_org_img_to_array(self):
        org_folder = self.config.testing_org_img_folder
        x = []
        for file in os.listdir(org_folder):
            path1 = os.path.join(org_folder, file)
            img1 = cv2.imread(path1, 1)
            img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
            img1 = cv2.resize(img1, (512, 512))
            x.append(img1)
        x = np.array(x)
        return x

    def convert_seg_img_to_array(self):
        seg_folder = self.config.testing_segment_img_folder
        y = []
        for file2 in os.listdir(seg_folder):
            path2 = os.path.join(seg_folder, file2)
            img2 = np.array(cv2.imread(path2, 1))
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
            img2 = cv2.resize(img2, (512, 512))
            y.append(img2)
        y = np.array(y)
        return y


    @staticmethod
    def rgb_to_2D_label(image):
        image_seg = np.zeros(image.shape, dtype= np.uint8)

        image_seg[np.all(image == np.array([255, 235, 0]), axis= -1)] = 0
        image_seg[np.all(image == np.array([0, 0 ,0]), axis= -1)] = 1
        image_seg[np.all(image == np.array([27 ,71, 151]), axis= -1)] = 2
        image_seg[np.all(image == np.array([111, 48, 253]), axis= -1)] = 3
        image_seg[np.all(image == np.array([137, 126, 126]), axis= -1)] = 4
        image_seg[np.all(image == np.array([201, 19, 223]), axis= -1)] = 5
        image_seg[np.all(image == np.array([254, 233, 3]), axis= -1)] = 6
        image_seg[np.all(image == np.array([255, 0, 29]), axis= -1)] = 7
        image_seg[np.all(image == np.array([255, 159, 0]), axis= -1)] = 8
        image_seg[np.all(image == np.array([255, 160, 1]), axis= -1)] = 9

        image_seg = image_seg[:,:,0]
        return image_seg
    
    
    def rgb_to_class_num(self):
        segArray = self.convert_seg_img_to_array()
        labels_list = []                             
        for i in range(segArray.shape[0]):
            label = self.rgb_to_2D_label(segArray[i])
            labels_list.append(label)
        labels_arr = np.array(labels_list)
        labels_arr = np.expand_dims(labels_arr, axis=3)
        return labels_arr
    
    def convert_to_categorical(self):
        imgArray = self.rgb_to_class_num()
        classes = self.config.number_of_classes
        label_categorical = to_categorical(imgArray, num_classes = classes)
        return label_categorical
    

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        custom_objects = {
            'bce_jaccard_loss': bce_jaccard_loss,
            'iou_score':iou_score
        }
        return tf.keras.models.load_model(path, custom_objects = custom_objects)

    
    def evaluation(self):
        model = self.load_model(self.config.model_path)
        org_img_arr = self.convert_org_img_to_array()
        y_cat = self.convert_to_categorical()
        self.score = model.evaluate(org_img_arr, y_cat)


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        

[2023-07-27 16:35:04,437: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-07-27 16:35:04,444: INFO: utils: NumExpr defaulting to 8 threads.]
Segmentation Models: using `keras` framework.


In [7]:


try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    eval_process = ModelEvaluation(config = evaluation_config)
    eval_process.evaluation()
    eval_process.save_score()
except Exception as e:
    raise e

[2023-07-27 16:35:05,777: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-27 16:35:05,777: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-27 16:35:05,783: INFO: common: created directory at: artifacts]
..........1
1/1 [==============================] - 1s 1s/step - loss: 2272.4985 - accuracy: 0.1665
................2
[2023-07-27 16:35:07,746: INFO: common: json file saved at: scores.json]


In [ ]:
# try:
#     config = ConfigurationManager()
#     data_process_config = config.get_data_processing_config()
#     data_process = ModelEvaluation(config = data_process_config)
# #     org_img_arr = data_process.convert_org_img_to_array()
#     print(org_img_arr.shape, "..........1")
# #     seg_img_arr = data_process.convert_seg_img_to_array()
# #     print(seg_img_arr.shape, "..........2")
# #     y_cls = data_process.rgb_to_class_num()
# #     print(y_cls.shape, "..........3")
# #     y_cat = data_process.convert_to_categorical()
# #     print(y_cat.shape, "..........4")
#     data_process.evaluation()
#     print('................2')
#     data_process.save_score()
# except Exception as e:
#     raise e